In [10]:
%load_ext autoreload
%autoreload 2

In [11]:
import numpy as np
import open3d as o3d
import copy

from utils import readXML, Intrinsics, Extrinsics, RGBD, crop_geometry, draw_registration_result, pick_points, manual_registration

In [23]:
xml_dir = "/home/yunzhu/Documents/project_phystouch/phystouch/data_recording/data"
xml_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/data/"

intrinsics = Intrinsics(xml_dir)
extrinsics = Extrinsics(xml_dir)
rgbds = RGBD(xml_dir, recording="recording_wAprilTag/20210714_003835")

# Stitching point clouds

### camera 020122061233

In [24]:
camera = "020122061233"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_1 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)

o3d.visualization.draw_geometries([pcd_1])

### camera 020122061651

In [25]:
camera = "020122061651"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_2 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)
o3d.visualization.draw_geometries([pcd_2])

### camera 821312060044

In [26]:
camera = "821312060044"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_3 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)
o3d.visualization.draw_geometries([pcd_3])

### camera 821312062243

In [27]:
camera = "821312062243"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_4 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)
o3d.visualization.draw_geometries([pcd_4])

### Stitched Point Cloud

In [28]:
pcds = pcd_1 + pcd_2 + pcd_3 + pcd_4

o3d.visualization.draw_geometries([pcds])

# ICP

In [29]:
obj_name = "005_tomato_soup_can"
obj_name = "003_cracker_box"
obj_name = "036_wood_block"


obj_path = f"C:/Users/lucas/Desktop/UPC/MIT/tactile2object/estimate_object_pose/ycb_converted/{obj_name}.ply"

obj_pcd = o3d.io.read_point_cloud(obj_path)
obj_mesh = o3d.io.read_triangle_mesh(obj_path)

obj_pcd.scale(0.001, center=(0,0,0))
obj_mesh.scale(0.001, center=(0,0,0))

TriangleMesh with 8194 points and 16384 triangles.

In [30]:
source = copy.deepcopy(obj_pcd)
target = copy.deepcopy(pcds)
transformation = manual_registration(source, target)

Manual ICP
Visualization of two point clouds before manual alignment

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D INFO] Picked point #7889 (0.074, 0.022, 0.2) to add in queue.
[Open3D INFO] Picked point #6937 (-0.0082, 0.041, 0.2) to add in queue.
[Open3D INFO] Picked point #2046 (-0.0073, 0.041, 0.00031) to add in queue.
[Open3D INFO] Picked point #2748 (0.075, 0.022, 0.00048) to add in queue.


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #928637 (0.089, -0.22, 0.6) to add in queue.
[Open3D INFO] Picked point #642911 (0.16, -0.18, 0.57) to add in queue.
[Open3

In [31]:
obj_mesh_t = copy.deepcopy(obj_mesh).transform(transformation)

obj_mesh_t.compute_vertex_normals()
obj_mesh_t.paint_uniform_color([1, 0.706, 0])

o3d.visualization.draw_geometries([pcds, obj_mesh_t])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [32]:
transformation

array([[ 0.7790807 , -0.07565391,  0.62234215, -0.01968236],
       [ 0.56767006,  0.50640622, -0.64907892, -0.05254062],
       [-0.26605257,  0.85896987,  0.43747777,  0.54361907],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])